In [ ]:
using Pkg
Pkg.activate(joinpath(@__DIR__,".."))

In [ ]:
using CalciumImagingAnalyses
using CalciumImagingAnalyses: get_cell_data, get_behavioural_data, decode, train_decoder

In [ ]:
CalciumImagingAnalyses.get_behavioural_data

## Decoding
We'll first construct a linear discriminant-based decoder to decode the direction that the mouse is going to poke. The input to the decoder will be the activity of all the calcium traces aligned to each poke event, averaged over a 1s window leading up to the poke. Because we have many more cells that trials, we'll first find a subspace spanned by the top singular vectors of the average activity matrix. We'll include enough singular vectors to account for 75% of the total weigth of the singular values. 

In [ ]:
datadir = joinpath(@__DIR__, "..","..","data")
# load data from the last day of pre-phase
cell_data_pre, timestamps_pre = get_cell_data(joinpath(datadir,"E10A","trace_10A_PRE3_unix.csv");filter_accepted=false)
# load behavioural data
pokes_pre = get_behavioural_data(joinpath(datadir, "E10A_E10B_Behavior/PRED3/E10A_PRED3.csv"))

# train a decoder to decode the mouse's chosen poke direction
q_pre = decode(cell_data_pre, timestamps_pre, pokes_pre.rewarded_poke_time, pokes_pre.rewarded_poke_type;pratio=0.75);

Let's plot the decoder output as a function of time, in the session, with the poke events indicated by blue vertical lines (left pokes) and right vertical lines (right pokes). 

In [ ]:
using CairoMakie

fig = CalciumImagingAnalyses.plot_timeseries(q_pre, timestamps_pre, pokes_pre.rewarded_poke_time, pokes_pre.rewarded_poke_type)

The first thing to observe is that this particular mouse has a strong preference for leftward pokes, even though both left and right pokes are being rewarded. Consistent with this, the value of the decoder function mostly stays above 0. Notably, It only seems to correctly decode 2 of 4 rightward pokes, while the left poke 7.5min is misclassified as a rigth poke (the decoder values dips sharply below 0). In addition, there doesn't seem to be a strong correlation between the dynamics of the decoder function value and the poke event; for some events, one can clearly observe an increase in the decoder value, while for other events, the decoder values appears to mostly flat.

Let's apply the decoder trained during the pre phase to the traces obtained during the associate fixed rate phase (aFR1). During this phase, only one of the ports is rewarded, and the mouse has to figure out through trial and error which port it is.

In [ ]:
pokes_afr = get_behavioural_data(joinpath(datadir,"E10A","behaviour/aFR1_D1/2023/11/FED002_112023_00.csv"))
cell_data_afr, timestamps_afr = get_cell_data(joinpath(datadir,"E10A","trace_10A_aFR1_unix.csv");filter_accepted=false)
# get the subspace w and the lda space from the pre-phase
w,lda,_ = train_decoder(cell_data_pre, timestamps_pre, pokes_pre.rewarded_poke_time, pokes_pre.rewarded_poke_type;pratio=0.75)

# project the data from the afr phase onto the subspace obtained from the pre-phase, and then apply the pre-phase plot_decoder
# combine both rewarded and unrewarded pokes
poke_time = [pokes_afr.rewarded_poke_time;pokes_afr.unrewarded_poke_time]
# label rewarded vs unrewarded
poke_types = [fill("rewarded", length(pokes_afr.rewarded_poke_time));fill("unrewarded", length(pokes_afr.unrewarded_poke_time))]
# label left vs right
poke_types2 = [pokes_afr.rewarded_poke_type;pokes_afr.unrewarded_poke_type]

q_afr = decode(w, lda, cell_data_afr, timestamps_afr, poke_time;pratio=0.75)
fig2 = CalciumImagingAnalyses.plot_timeseries(q_afr, timestamps_afr,poke_time, collect(zip(poke_types, poke_types2)))

Since this mouse had a strong preference for left pokes, and the aFR phase reinforces this bias, the decoder appears to do quite well also here. Note, though, that that is because the decoder appears to simply output 'left' (i.e. positive value) most of the time, rendering it correct almost by default. Also note that, there is negative deflection at the very beginning of the sesison whic is followed by a left, rather than right, poke, follow by a strong postive deflection which is followed by two rigth pokes in rapid succession. For both of these cases, the decoder makes a mistake. 

We would need to train a decoder on session where we also see a number of right rewarded choices to have confidence that it is picking something real in the data. This can be achieved by combining the aFR and rFR phases, i.e. the associative and the reversal phase. For the mouse shown above, the rFR phase would consist of trials in which only right pokes were rewarded, and these trials would allow the decoder to get a good representation of what the calcium activity assicated with right pokes looks like.